In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/dataset/images'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Let's get the libraries
Fastai by Jeremy Howard https://www.fast.ai/about/

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch.nn.functional")

In [ ]:
from fastai import *
from fastai.vision import *

In [ ]:
path = Path("../input/dataset/images")

### The dataset has 4 labels and I only got 100 (total of 400) images for each class to check the performance with less data ###

In [ ]:
classes = ['bee', 'other_insect', 'wasp', 'other_noinsect']

### Here's the ImageDataBunch object to put together all the train/test/validation data in order to have the necessary transformations to the images ###

In [ ]:
import numpy as np
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
                                  ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)

In [ ]:
data.classes

In [ ]:
data.show_batch(rows=4, figsize=(7,8))

### 4 labels, 320 images for training, 80 images for validation ###

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

### Using the Resnet34 model (a pre-trained model) ###

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=[error_rate, accuracy])

In [ ]:
learn.fit_one_cycle(3)

In [ ]:
learn.model_dir='/kaggle/working' 
learn.save("stage_0.bee_wasp")

In [ ]:
learn.load('stage_0.bee_wasp')

In [ ]:
learn.unfreeze()

### Getting the learning rate ###

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

### Choosing a good learning rate increased the accuracy to 96,25%!!!

In [ ]:
learn.fit_one_cycle(2, max_lr=slice(3e-5,3e-4))

In [ ]:
learn.model_dir='/kaggle/working' 
learn.save("stage_02.bee_wasp")

### Confusion Matrix ###

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

### That's it!! ###